In [ ]:
import numpy as np

In [ ]:
import optimization.optim_tools as optim_tools

kwargs = optim_tools.bisect_max(0, 1, None, None, None,
           bisection_tol=1e-3, solver=None, bisect_verbose=False, max_iters=1100, warm_start=True)
kwargs['max_iters'] = 2000
kwargs

In [ ]:
def svd_inverse(A):
    """ Inverts a matrix using singular value decomposition (SVD).
        The matrix doesn't have to be square or of full rank.
    """
    # A = U*(I*s)*V => is easier to invert (see literature)
    U, s, V = np.linalg.svd(A, full_matrices=False)  # SVD decomposition of A
    for i in xrange(0, len(s)):
        if s[i] > 0.001:
            s[i] = 1/s[i]        # ToDo: damping near singularities
    return np.dot(np.dot(V.T, np.diag(s)), U.T)

In [ ]:
def svd_solve(A, b):
    U, s, V = np.linalg.svd(A, full_matrices=False)  # SVD decomposition of A
    for i in xrange(0, len(s)):
        if s[i] > 0.001:
            s[i] = 1/s[i]        # ToDo: damping near singularities
    return V.T.dot(np.diag(s).dot(U.T.dot(b)))

In [ ]:
def svd_solve2(A, b):
    U, s, V = np.linalg.svd(A, full_matrices=False)  # SVD decomposition of A
    s[s > 0.001] = 1/s[s > 0.001]
    return V.T.dot(np.diag(s).dot(U.T.dot(b)))

In [ ]:
n=6
m=7
A_m = np.random.rand(n,m)
y_m = np.random.rand(n)
np.random.seed(0)

In [ ]:
%%timeit

A_m_inv = svd_inverse(A_m)
dq = np.dot(A_m_inv, y_m)

In [ ]:
%%timeit

dq = svd_solve(A_m, y_m)

In [ ]:
%%timeit

dq = svd_solve2(A_m, y_m)

In [ ]:
#import numpy as np
#import control as con
#from numpy import linalg as LA

import cvxpy
#import optimization.optim_tools as optim_tools#own file with helper

In [ ]:
import rospy
from tf import transformations as trans

R0 = trans.quaternion_matrix([0.500, -0.500, -0.500, -0.500])
t0 = trans.translation_matrix((-0.187, 0.359, 0.403))
T0 = trans.concatenate_matrices(t0, R0)

R1 = trans.quaternion_matrix([-0.323, 0.323, 0.629, 0.629])
t1 = trans.translation_matrix((-0.187, 0.330, 0.393))
T1 = trans.concatenate_matrices(t1, R1)

R2 = trans.quaternion_matrix([-0.208, 0.406, 0.792, 0.406])
t2 = trans.translation_matrix((-0.158, 0.335, 0.386))
T2 = trans.concatenate_matrices(t2, R2)

print T0
print T1
print T2

In [ ]:
import numpy as np
import cvxpy

n = 4

T_tool = cvxpy.Semidef(n, n) # Implys (459) (semidefinite for numerical reasons?)
T_obj = cvxpy.Semidef(n, n) # Implys (459) (semidefinite for numerical reasons?)
T_err0 = cvxpy.Semidef(n, n) # Implys (459) (semidefinite for numerical reasons?)
T_err1 = cvxpy.Semidef(n, n) # Implys (459) (semidefinite for numerical reasons?)
T_err2 = cvxpy.Semidef(n, n) # Implys (459) (semidefinite for numerical reasons?)

constraints = [T0*T_tool == T_obj,
               T1*T_tool == T_obj,
               T2*T_tool == T_obj,
              ]

In [ ]:

# Objective representation
obj = cvxpy.Minimize(cvxpy.trace(T_obj)) # Identical to geo_mean (in term of convexity and result)
#obj = cvxpy.Maximize(cvxpy.log_det(Q)) # Identical to geo_mean (in term of convexity and result)
#obj = cvxpy.Minimize(-cvxpy.geo_mean(Q)) # Not yet implemented

prob = cvxpy.Problem(obj, constraints)
prob.solve(verbose=True)

In [ ]:
import sympy as sp
import numpy as np
import numpy.linalg as la

w = sp.symbols('w_1')
s = sp.symbols('s_1')
b = sp.symbols('b_1:3')
d = sp.symbols('d_1')
x = sp.symbols('x_1:3')
c = sp.symbols('c_1:3')

u = sp.symbols('u_1')

X = sp.Matrix([x]).T
U = sp.Matrix([u])


A = sp.Matrix([[s, w],[-w,s]])
B = sp.Matrix([b]).T
C = sp.Matrix([c])
D = sp.Matrix([d])

AA = sp.lambdify((s, w), A)
BB = sp.lambdify(b, B)
CC = sp.lambdify(c, C)
DD = sp.lambdify(d, D)

z = X.T.row_join(sp.Matrix([s])).row_join(sp.Matrix([w])).row_join(B.T).row_join(C).row_join(D).T
#print z

fx = A*X+B*U
f = sp.Matrix([0 for _ in range(len(z)-len(x))])
f = fx.col_join(f)

print "f="; sp.pprint(f)
ff = sp.lambdify(z.T.row_join(U), f)

print "F="; sp.pprint(f.jacobian(z))#; print len(f.jacobian(z))
FF = sp.lambdify(z.T.row_join(U), f.jacobian(z))

h = C*X+D*U
print "h="; sp.pprint(h)
hh = sp.lambdify(z.T.row_join(U), h)

print "H="; sp.pprint(h.jacobian(z))#; print len(h.jacobian(z))
HH = sp.lambdify(z.T.row_join(U), h.jacobian(z))

#sp.pprint((h.jacobian(z)*h.jacobian(z).T))

In [ ]:
z_test = [0, 0, -1, 1, -2, 1, 0.1, 0.2, 0]
zu_test = np.matrix([0, 0, -1, 1, -2, 1, 0.1, 0.2, 0, 2]).T

print zu_test
#print ff(*(zu_test.tolist()[0]))

def _c(M):
    return M.tolist()[0]

print ff(*_c(zu_test))


In [ ]:
rho = 0.1


#R = np.eye(len(z))
R = 1
#print np.diag([sym for sym in z])
print R

q = np.array([])
for sym in z:
    if sym in X.T.row_join(C):
        q = np.hstack((q, 0))
    else:
        q = np.hstack((q, 1))

Q = rho * np.matrix(np.diag(q))
print Q

In [ ]:
### Testsystem ####
def calc_TestSystem(x, u):
    
    Af = np.matrix([[  2.84217094e-14,   1.71379179e+01],
                    [ -1.00000000e+02,  -1.85369064e+02]])

    Bf = np.matrix([[ 17.34396868],
                    [  9.87641374]])

    Cf = np.matrix([[ 0., -1.]])

    Df = np.matrix([[ 0.]])
    
    #print Af
    #print x
    #print "res"
    #print Af.dot(x)
    #print Bf.dot(u)
    #print "res end"
    x_dot = Af.dot(x) + Bf.dot(u)
    y = Cf.dot(x) + Df.dot(u)
    
    #print x_dot
    return y, x_dot

In [ ]:
#%time
# initial
# A
s_k0 = np.matrix([-1]) # real Teil
w_k0 = np.matrix([1])  # (+/-) imag Teil

# b
b_k0 = np.matrix([-2, 1])

# c
c_k0 = np.matrix([0.1, 0.2])

# d
d_k0 = np.matrix([0])


P_k0 = np.eye(len(z))

x_k0 = np.matrix([0, 0]).T
#print x_k0
u_k0 = np.matrix([1])

T = 0.001


# First values
x_k = x_k0
u_k = u_k0
z_k = np.hstack((x_k0.T, s_k0, w_k0, b_k0, c_k0, d_k0)).T
#print "z_k:", z_k

P_k = P_k0

# Loop start
for c, t in enumerate(np.arange(0, 10, T)):

    #from IPython.core.debugger import Tracer; Tracer()() 
    
    #print "x_k:\n", x_k
    y_k, x_dot = calc_TestSystem(x_k, u_k)
    #print "y_k:\n", y_k
        
    ##### Reconstruction from z
    #print "z_k:", z_k    
    
    x_k = np.matrix(z_k.T[0, 0:2], dtype=np.float).T
    #print "x_k:\n",x_k
    s_k = np.matrix(z_k.T[0, 2], dtype=np.float)
    #print s_k
    w_k = np.matrix(z_k.T[0, 3], dtype=np.float)
    #print w_k
    b_k = np.matrix(z_k.T[0, 4:6], dtype=np.float)
    #print b_k
    c_k = np.matrix(z_k.T[0, 6:8], dtype=np.float)
    d_k = np.matrix(z_k.T[0, 8], dtype=np.float)

    if c%1000==1: 
        print "Loop", t
        #print "s_k:\n", s_k
        #print "w_k:\n", w_k
        #print "b_k:\n", b_k
        #print "c_k:\n", c_k
        #print "d_k:\n", d_k
    
    ## System
    A_k = AA(s_k, w_k)
    B_k = BB(*_c(b_k))
    C_k = CC(*_c(c_k))
    D_k = DD(*_c(d_k))


    ##### Evaluation
    # State space
    #dx_k = A_k.dot(x_k) + B_k.dot(u_k)
    #y_k = C_k.dot(x_k) + D_k.dot(u_k)

    # Concate values for lambdafied Jacobians
    zu_k = np.hstack((z_k.T, u_k))
    #print "zu_k:\n", zu_k
    
    h_k = np.matrix(hh(*_c(zu_k))) # h = y_k (predicted)
    #print "h_k:\n", h_k
    H_k = np.matrix(HH(*_c(zu_k))) # h.Jacobian
    #print "H_k:\n", H_k

    
    f_k = np.matrix(ff(*_c(zu_k))) # f
    #print "f_k:\n", f_k
    F_k = np.matrix(FF(*_c(zu_k))) #f.Jacobian
    #print "F_k:\n", F_k
    
    ##### Kalman Filter

    # Prediction

    #print "P_k:\n", P_k
    #print "H_k.T:\n", H_k.T
    
    K_k = P_k.dot(H_k.T).dot(la.inv(H_k.dot(P_k).dot(H_k.T) + R))
    #print "K_k:\n", K_k

    Ps_k = (np.eye(len(z)) - K_k.dot(H_k)).dot(P_k) # P*_k
    #print "Ps_k:\n", Ps_k

    # Correction
    P_k1 = Ps_k + T*(F_k.dot(Ps_k) + Ps_k.dot(F_k.T) + Q)
    #print "P_k1:\n", P_k1

    z_k1 = (z_k + T*f_k + K_k.dot((y_k - h_k)))
    #print "z_k1:", z_k1

    # State Propagation
    x_k = x_k + x_dot*T
    #print "x_k:\n",x_k
    
    z_k = np.matrix(z_k1)
    #print "z_k:\n", z_k
    
    P_k = np.matrix(P_k1).astype(np.float)
    #print "P_k:\n", P_k
    

In [ ]:
print A_k.squeeze()
print b_k

In [ ]:
import control as con
ss = con.matlab.ss(A_k.squeeze(), b_k.T, c_k, d_k)
ss.pole()

In [ ]:
Af = np.matrix([[  2.84217094e-14,   1.71379179e+01],
                [ -1.00000000e+02,  -1.85369064e+02]])

Bf = np.matrix([[ 17.34396868],
                [  9.87641374]])

Cf = np.matrix([[ 0., -1.]])

Df = np.matrix([[ 0.]])

ss2 = con.matlab.ss(Af, Bf, Cf, Df)
ss2.pole()